In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os 
import subprocess as sp 
import shutil
from pathlib import Path
import MDAnalysis as mda
import MDAnalysisTests as mdat
import MDAnalysis.transformations as trans
import nglview as nv

import Package.helpers as helpers
import Package.organizer as organizer
import Package.NAMD as NAMD
import Package.MDA as MDA

MDA.empty_universe

In [2]:
empty = MDA.MDA.empty_universe(n_residues=10)

/home/pbuser/miniconda3/envs/work/lib/python3.9/site-packages/MDAnalysis/core/universe.py:433: UserWarning: Residues specified but no atom_resindex given.  All atoms will be placed in first Residue.
  warnings.warn(
/home/pbuser/miniconda3/envs/work/lib/python3.9/site-packages/MDAnalysis/core/universe.py:439: UserWarning: Segments specified but no segment_resindex given.  All residues will be placed in first Segment
  warnings.warn(


MDA.create_waterverse

In [8]:
waters=10
n_residues = waters
n_atoms = n_residues * 3
resindices = np.repeat(range(n_residues), 3)
assert len(resindices) == n_atoms
segindices = [0] * n_residues
waterverse = MDA.MDA.empty_universe(n_residues=n_residues)


In [9]:
waterverse

<Universe with 30 atoms>

In [ ]:
waterverse.add_TopologyAttr('name', ['O', 'H1', 'H2']*n_residues)
waterverse.add_TopologyAttr('type', ['O', 'H', 'H']*n_residues)
waterverse.add_TopologyAttr('resname', ['SOL']*n_residues)
waterverse.add_TopologyAttr('resid', list(range(1, n_residues+1)))
waterverse.add_TopologyAttr('segid', ['SOL'])
#positions can be simply assigned:
h2o = np.array([[ 0,        0,       0      ],  # oxygen
                [ 0.95908, -0.02691, 0.03231],  # hydrogen
                [-0.28004, -0.58767, 0.70556]]) # hydrogen
grid_size, spacing, coordinates = 10, 8, []
# translating h2o coordinates around a grid
for i in range(n_residues):
    x = spacing * (i % grid_size)
    y = spacing * ((i // grid_size) % grid_size)
    z = spacing * (i // (grid_size * grid_size))
    xyz = np.array([x, y, z])
    coordinates.extend(h2o + xyz.T)
coord_array = np.array(coordinates)
assert coord_array.shape == (n_atoms, 3)
waterverse.atoms.positions = coord_array
assert not hasattr(sol, 'bonds')
bonds = []
for o in range(0, n_atoms, 3):
    bonds.extend([(o, o+1), (o, o+2)])
waterverse.add_TopologyAttr('bonds', bonds)

In [ ]:
waterverse = create_waterverse(waters=10)

filling universe with water.


AttributeError: module 'Package.MDA' has no attribute 'empty_universe'

MDA.add_segment

In [ ]:
TIP3P = MDA.MDA.add_segment(universe = waterverse, segID='TIP3P', residue_selection='all')

Assuming all atoms are already in the universe, this function just splits existing atoms into new segments. Residues can not be split among two segments.


MDA.tile_universe

In [ ]:
#TIP3P_tiled = MDA.MDA.tile_universe(universe=TIP3P, n_x=100, n_y=100, n_z=100)

In [ ]:
print(TIP3P.dimensions)

[73.23912 73.23912 73.23912 90.      90.      90.     ]


MDA.import_structure

In [ ]:
#MDA.MDA.import_structure(structure='cco_and_water.pdb', waterverse=TIP3P)

In [ ]:
#import MDAnalysis as mda
#structure = 'cco_and_water.pdb'
#waterverse = waterverse
#import protein
#protiverse = mda.Universe(structure)
#center around origin
#water_cog_ = waterverse.atoms.center_of_geometry()
#waterverse.atoms.positions -= water_cog_
#water_cog = waterverse.atoms.center_of_geometry()
#protein_cog_ = protiverse.atoms.center_of_geometry()
#protiverse.atoms.positions -= protein_cog_
#protein_cog = protiverse.atoms.center_of_geometry()
#print('Water Center of geometry '+ water_cog)
#print('Protein Center of geometry '+ protein_cog)
#for pos in water_cog:
#    if pos > 0.1:
#        print('Aligning origins resulted in unusually large values. Please consider looking at the center of geometry operations of the waterverse.')
#for pos in protein_cog:
#    if pos > 0.1:
#        print('Aligning origins resulted in unusually large values. Please consider looking at the center of geometry operations of the protiverse.')
#create merged universe
#universe = mda.Merge(protiverse.atoms, waterverse.atoms)
#find the size of our universe as 1.5 times the largest size in one direction
#dist = 1.5*(max(u.atoms.positions[:, 0]) - min(u.atoms.positions[:, 0]))
#fix the size of our universe, 90 for 90° angles (box-shaped universe)
#universe.dimensions = [dist, dist, dist, 90, 90, 90]
#tile out the universe (explicit PBCs)
#if kwargs.get('tile') is not None:
#    if kwargs.get('n_x') is not None:
#        if kwargs.get('n_y') is not None:
#            if kwargs.get('n_z') is not None:
#                universe = MDA.tile_universe(universe=universe, n_x=n_x, n_y=n_y, n_z=n_z)
#    else:
#        sys.exit('Tiling out the system was requested, but no specifications for tiling have been given. Please enter n_x, n_y, n_z values as function arguments.')

In [ ]:
empty = MDA.MDA.empty_universe(n_residues=100)
waterverse_ = MDA.MDA.create_waterverse(waters=10)
waterverse__ = MDA.MDA.create_waterverse(waters=10)

Number of atoms (n_atoms) is unknown and guessed to be for a universe to be filled with water. Therefore, n_atoms will be n_residues*3.
Filling universe with water.
Number of atoms (n_atoms) is unknown and guessed to be for a universe to be filled with water. Therefore, n_atoms will be n_residues*3.
Filling universe with water.
Number of atoms (n_atoms) is unknown and guessed to be for a universe to be filled with water. Therefore, n_atoms will be n_residues*3.


In [ ]:
universes = []
universes.append(empty.atoms)
universes.append(waterverse_.atoms)
universes.append(waterverse__.atoms)
#waterverse___ = mda.Merge(*universes)

MDA.MDA.align_structures

In [ ]:
protiverse=mda.coordinates.PDB.PDBReader('cco_and_membrane.pdb')
#waterverse=
#universe = MDA.MDA.align_structures(protiverse=cco_and_membrane.pdb, waterverse=water.pdb)
protiverse#=mda.Universe(protiverse)

<PDBReader cco_and_membrane.pdb with 1 frames of 37932 atoms>

In [ ]:
#MDA.MDA.tile_universe(universe=waterverse, n_x=75, n_y=75, n_z=75)
#tile_universe is written so suboptimally, that it routinely crashes my computer.